In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split



In [50]:
class MyModels:
  def __init__(self,data,lables, n_mfcc=40  ): 
    self.data = data
    self.lables = lables
    self.n_mfcc = n_mfcc

  def train_test(self):
        X_train, X_test, lables_train, lables_test = train_test_split(self.data, self.lables, test_size=0.1, random_state=42,stratify=lables['emotion'])
        return X_train, X_test, lables_train, lables_test
  

  def conv_model(self,input_shape1):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv1D(32, kernel_size=(5),padding ='same', input_shape=input_shape1))
    model.add(tf.keras.layers.Activation(tf.keras.activations.relu))
    model.add(tf.keras.layers.Conv1D(64, kernel_size=(5),padding ='same'))
    model.add(tf.keras.layers.Activation(tf.keras.activations.relu))

    model.add(tf.keras.layers.MaxPooling1D(pool_size=(5)))
    model.add(tf.keras.layers.Dropout(0.4))
    
    model.add(tf.keras.layers.Conv1D(64, kernel_size=(5),padding ='same',kernel_regularizer=tf.keras.regularizers.l2(0.01), bias_regularizer=tf.keras.regularizers.l2(0.01)))
    model.add(tf.keras.layers.Activation(tf.keras.activations.relu))

    model.add(tf.keras.layers.MaxPooling1D(pool_size=(2)))
    model.add(tf.keras.layers.Dropout(0.4))

    model.add(tf.keras.layers.Flatten())
    #model.add(tf.keras.layers.Dense(128, activation='relu'))
    #model.add(tf.keras.layers.Dropout(0.4))

    model.add(tf.keras.layers.Dense(128,activation=tf.keras.activations.relu))
    #model.add(tf.keras.layers.Dense(32,activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dropout(0.4))


    model.add(tf.keras.layers.Dense(8,activation=tf.keras.activations.softmax))

    model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001),loss =tf.keras.losses.CategoricalCrossentropy() ,metrics=(['acc']))
    model.summary()
        
    return model


In [22]:

data = pd.read_csv('data_frame.csv')
lables = pd.read_csv('lables_data_frame.csv')


data = np.array(data)



In [15]:
print(data.shape)



(2452, 40)


In [51]:
model = MyModels(data,lables)
X_train, X_test, lables_train, lables_test = model.train_test()
    
y_train = lables_train.iloc[:,-1]
y_train = tf.keras.utils.to_categorical(y_train-1, dtype ="uint8") 
    
y_test = lables_test.iloc[:,-1]
y_test = tf.keras.utils.to_categorical(y_test-1, dtype ="uint8") 

input_shape = (X_train.shape[1],1)

In [17]:
print(X_train.shape,X_test.shape)
print(y_train.shape)

(2206, 40) (246, 40)
(2206, 8)


In [52]:
cnn_model = model.conv_model(input_shape)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_14 (Conv1D)           (None, 40, 32)            192       
_________________________________________________________________
activation_6 (Activation)    (None, 40, 32)            0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 40, 64)            10304     
_________________________________________________________________
activation_7 (Activation)    (None, 40, 64)            0         
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 8, 64)             0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 8, 64)             0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 8, 64)            

In [53]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)

X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)


In [54]:
cnn_model.fit(X_train,y_train,epochs=2000,batch_size = 80,shuffle = True,validation_data=(X_test,y_test))


Epoch 1/2000
28/28 [==============================] - 1s 23ms/step - loss: 19.0282 - acc: 0.1263 - val_loss: 4.3696 - val_acc: 0.1545
Epoch 2/2000
28/28 [==============================] - 0s 16ms/step - loss: 13.5338 - acc: 0.1491 - val_loss: 3.4723 - val_acc: 0.1545
Epoch 3/2000
28/28 [==============================] - 0s 15ms/step - loss: 10.1376 - acc: 0.1234 - val_loss: 3.0339 - val_acc: 0.1463
Epoch 4/2000
28/28 [==============================] - 0s 16ms/step - loss: 7.9673 - acc: 0.1170 - val_loss: 2.8435 - val_acc: 0.1260
Epoch 5/2000
28/28 [==============================] - 0s 16ms/step - loss: 6.3305 - acc: 0.1427 - val_loss: 2.7772 - val_acc: 0.1504
Epoch 6/2000
28/28 [==============================] - 0s 17ms/step - loss: 5.5774 - acc: 0.1282 - val_loss: 2.7215 - val_acc: 0.1423
Epoch 7/2000
28/28 [==============================] - 0s 16ms/step - loss: 4.5684 - acc: 0.1437 - val_loss: 2.7068 - val_acc: 0.1382
Epoch 8/2000
28/28 [==============================] - 0s 16ms/step

In [55]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:

import os
name = 'model2.h5'
loc = 'drive/MyDrive/project'

if not os.path.isdir(loc):
    os.makedirs(loc)


#model_path = os.path.join(loc, name)
cnn_model.save(loc + '/' + name)


In [57]:
y_pred = cnn_model.predict(X_test)
y_pred_raw = np.argmax(y_pred, axis=1)

    
y_test_raw = np.argmax(y_test, axis=1)
    
print("test accuracy: ",sum(y_pred_raw == y_test_raw)/len(y_test_raw))

test accuracy:  0.8130081300813008


In [58]:
good = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0}

for i in range(len(y_pred_raw)):
  if y_pred_raw[i]==y_test_raw[i]:
    good[y_pred_raw[i]] += 1
  
vals,freq = np.unique(y_test_raw,return_counts=True)

print(good)

real = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0}

for i in range(len(freq)):
  real[i] = freq[i]

print(real)



emotions = {0 : 'neutral', 1 : 'calm', 2 : 'happy', 3 : 'sad',4 : 'angry', 5 : 'fearful', 6 : 'disgust', 7 : 'surprised'}
           
acc = {}
for i in range(len(good)):
  acc[emotions[i]] = good[i]/real[i]

print('accuracy per feeling: ' ,acc)

{0: 13, 1: 32, 2: 32, 3: 27, 4: 33, 5: 34, 6: 15, 7: 14}
{0: 19, 1: 38, 2: 37, 3: 38, 4: 38, 5: 38, 6: 19, 7: 19}
accuracy per feeling:  {'neutral': 0.6842105263157895, 'calm': 0.8421052631578947, 'happy': 0.8648648648648649, 'sad': 0.7105263157894737, 'angry': 0.868421052631579, 'fearful': 0.8947368421052632, 'disgust': 0.7894736842105263, 'surprised': 0.7368421052631579}
